In [ ]:
# depndencies
import pandas as pd
import requests
from splinter import Browser
from bs4 import BeautifulSoup as bs
from pprint import pprint
import time
import re

# CANADIAN INDEX OF WELLBEING

In [ ]:
# prepare chromedriver
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

# set browser with url of Canadian Index of Wellbeing
url1 = 'https://uwaterloo.ca/canadian-index-wellbeing/reports/2016-canadian-index-wellbeing-national-report/executive-summary'
browser.visit(url1)
time.sleep(5)

# set up parser
html = browser.html
soup = bs(html, 'lxml')

In [ ]:
url = 'https://uwaterloo.ca'

In [7]:
# get images
image1_url = url + soup.find_all('img')[0]['src']
image2 = soup.find_all('img')[1]['src']
image2_url = url + image2
image1_url

'https://uwaterloo.ca/canadian-index-wellbeing/sites/ca.canadian-index-wellbeing/files/resize/uploads/images/c011676-ciw-nationalreport-ciwvsgdp-750x500.jpg'

In [9]:
#click open table
#browser.find_link_by_partial_text('DATA TABLE').click()
#time.sleep(5)

In [29]:
# use pandas to read the tables
tables = pd.read_html(url1)
tables

[      Year GDP per capita           CIW
 0   1994.0     0.00  1.73   0.00  -0.73
 1   1996.0     2.22  5.53  -1.41  -1.23
 2   1998.0    8.62  13.65  -1.06  -0.50
 3   2000.0   18.20  18.36   -0.29  1.06
 4   2002.0   20.42  21.59    2.35  3.80
 5   2004.0   24.10  26.80    4.56  5.77
 6   2006.0   29.09  30.36    7.21  8.39
 7   2008.0   29.87  25.25    8.84  8.50
 8   2010.0   30.87  34.05    6.43  5.91
 9   2012.0   34.81  36.21    6.82  8.18
 10  2014.0          37.99          9.93
 11     NaN           38.0           9.9,
                  Year  CIW  Community Vitality  Democratic Engagement  \
 0                1994  0.0                 0.0                    0.0   
 1                1995 -0.7                 0.1                   -0.7   
 2                1996 -1.4                 0.2                   -1.3   
 3                1997 -1.2                 0.1                   -2.0   
 4                1998 -1.1                 0.0                   -2.7   
 5                1999

In [44]:
t1=tables[0].copy()
t1 = t1.drop(t1.index[[10,11]])
t1['Year'] = t1['Year'].astype(int)
t1['Year1']=t1['Year']+1
t1["GDP percapita increase"]= t1["GDP per capita"].str.split(" ", n = 1, expand = True)[1].astype(float)
t1["CIW increase"]= t1["CIW"].str.split(" ", n = 1, expand = True)[1].astype(float)
t1=t1.drop(t1.columns[[0,1,2]], axis=1).set_index('Year1')
t1

,GDP percapita increase,CIW increase
Year1,,
1995,1.73,-0.73
1997,5.53,-1.23
1999,13.65,-0.50
2001,18.36,1.06
2003,21.59,3.80
2005,26.80,5.77
2007,30.36,8.39
2009,25.25,8.50
2011,34.05,5.91


In [50]:
vsGDP_Table=tables[0].copy()
vsGDP_Table = vsGDP_Table.drop(vsGDP_Table.index[11])
vsGDP_Table['Year'] = vsGDP_Table['Year'].astype(int)
vsGDP_Table["GDP percapita increase"]= vsGDP_Table["GDP per capita"].str.split(" ", n = 1, expand = True)[0].astype(float)
vsGDP_Table["CIW increase"]= vsGDP_Table["CIW"].str.split(" ", n = 1, expand = True)[0].astype(float)
vsGDP_Table = vsGDP_Table.drop(vsGDP_Table.columns[[1,2]], axis=1).set_index('Year')
vsGDP_Table = pd.concat([vsGDP_Table, t1]). sort_index()
vsGDP_Table

,GDP percapita increase,CIW increase
1994,0.00,0.00
1995,1.73,-0.73
1996,2.22,-1.41
1997,5.53,-1.23
1998,8.62,-1.06
1999,13.65,-0.50
2000,18.20,-0.29
2001,18.36,1.06
2002,20.42,2.35
2003,21.59,3.80


In [32]:
CIW_Table=tables[1].copy().set_index('Year')
CIW_Table.drop(CIW_Table.index[21], inplace=True)
CIW_Table.rename(columns={'Community Vitality': 'com_vty', 'Democratic Engagement': 'dem_egmt','Education': 'edu',
                          'Environment': 'env','Healthy Populations': 'hlty_pop', 'Leisure and Culture': 'leisure',
                          'Living Standards': 'liv_std', 'Time Use': 'time_use',},inplace=True)
CIW_Table

,CIW,com_vty,dem_egmt,edu,env,hlty_pop,leisure,liv_std,time_use
Year,,,,,,,,,
1994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1995,-0.7,0.1,-0.7,-1.7,-0.8,-1.2,0.5,0.2,-2.3
1996,-1.4,0.2,-1.3,-3.1,-0.2,-2.0,0.7,-1.4,-4.1
1997,-1.2,0.1,-2.0,-2.7,-0.7,-1.8,1.1,-0.4,-3.4
1998,-1.1,0.0,-2.7,-2.9,-2.7,-1.5,0.9,4.3,-3.9
1999,-0.5,1.3,-3.4,-1.2,-2.1,-7.0,0.3,10.0,-1.9
2000,-0.3,2.2,-4.1,0.7,-4.3,-9.5,0.7,11.9,0.1
2001,1.1,3.8,-0.9,2.9,-4.2,-10.2,-2.1,18.8,0.5
2002,2.4,5.8,2.1,5.9,-3.6,-7.1,-2.1,17.3,0.5
